In [ ]:
!pip install QuantumRingsLib
!pip install quantumrings-toolkit-qiskit

!pip uninstall -y qiskit qiskit-aer qiskit-terra
!pip install qiskit==1.3.1 qiskit-aer --no-cache-dir


!pip install python-sat

In [ ]:
import numpy as np
import math
from sympy import gcd
from pysat.solvers import Glucose3  # SAT solver
# Import the required modules
from qiskit.circuit import QuantumCircuit
from qiskit import QuantumCircuit, transpile, QuantumRegister, ClassicalRegister, AncillaRegister
from qiskit.visualization import plot_histogram
from matplotlib import pyplot as plt

import QuantumRingsLib
from QuantumRingsLib import QuantumRingsProvider
from quantumrings.toolkit.qiskit import QrBackendV2
from quantumrings.toolkit.qiskit import QrJobV1

from matplotlib import pyplot as plt
import random

# Load semiprimes (semiprime numbers)
# semiprimes = {
#     8: 143, 10: 899, 12: 3127, 14: 11009, 16: 47053, 18: 167659, 20: 744647, 22: 3036893,
#     24: 11426971, 26: 58949987, 28: 208241207, 30: 857830637, 32: 2776108693, 34: 11455067797,
#     36: 52734393667, 38: 171913873883, 40: 862463409547, 42: 2830354423669, 44: 12942106192073,
#     46: 53454475917779, 48: 255975740711783, 50: 696252032788709, 52: 3622511636491483,
#     54: 15631190744806271, 56: 51326462028714137, 58: 217320198167105543, 60: 827414216976034907,
#     62: 3594396771839811733, 64: 13489534701147995111, 66: 48998116978431560767, 68: 220295379750460962499,
#     70: 757619317101213697553, 72: 4239706985407101925109, 74: 13081178794322790282667
# }

semiprimes = {
    76: 48581232636534199345531,
    78: 263180236071092621088443,
    80: 839063370715343025081359,
    82: 3145102596907521247788809,
    84: 13410747867593584234359179,
    86: 74963308816794035875414187,
    88: 196328049947816898123437813,
    90: 900212494943030042797046797,
    92: 3408479268382267351010110507,
    94: 13410207519922000104514406009,
    96: 56540697284955642837798912007,
    98: 212736089539904961817389577063,
    100: 793334180624272295351382130129,
    102: 3680428259937415378335285504779,
    104: 16332602937710799037362680335351,
    106: 57831217106245162293092061499909,
    108: 248282609001868585591158742505413,
    110: 1052209925061489272435010659874991,
    112: 3787041098266201578641927407194191,
    114: 13439273072951974276382903784255637,
    116: 53125759235002964322304682027959157,
    118: 291658670780267526191653438575411491,
    120: 1153151809972770124185028131269906357,
    122: 4932188172696454339739318297923925849,
    124: 17356117513473888567381746939296160477,
    126: 70736588847140822442757134113822365169,
    128: 212537944946563703298274772990879967689,
    130: 1084744344991496578890012624928207712213,
    132: 4047187468515523114449296638529157815821,
    134: 17544816517388983161547445055372483694669,
    136: 68909661794716346033524625875715931123259,
    138: 260300599328718051119938934425302978167179,
    140: 999828801740135651717021198616667990536367,
    142: 3355483482297520282820961102046248621459173,
    144: 16631837140942852587293950352766905297528897,
    146: 85937541327755603528549497212629785145477713,
    148: 244379668284326186252863240169712092438850161,
    150: 1063901349880514705720387916171354938099705083,
    152: 4094301101616740933345553372923577469734725267,
    154: 18540363865879151504375215350972044662106182549,
    156: 74276654746024856066871733818210517618023005407,
    158: 275275419272426358674813929221957447870184635413,
    160: 1014737718417810170242113026859620408620227447581,
    162: 4478072308913255100498385793781543335816717273939,
    164: 16727777314454032963341593941190650285738385202361,
    166: 69590306463255114448417962649373740941080626000523,
    168: 315687739381804055710020885538727496795372933617227,
    170: 1267155784800316161358824961914132766229147990344997,
    172: 4010354160858905534730018630654476125957281838169269,
    174: 14153584803038660862653346883739092747954923774660251,
    176: 77432705949923513909636069140456666815340352624073251,
    178: 339852994275131877601341971143799720360724856306184203,
    180: 1179359789143995137143701718442354039063066637950737429,
    182: 4919304328219861879269200803487336608500936862015264459,
    184: 17505199067294797183746989001423060541688565870542696723,
    186: 69942997811183689049499239546127943802272331478524805397,
    188: 261274809295595261399973998023978896104786545957568559137,
    190: 1406369495880278908988444017973006804779746131711977427971,
    192: 5096174527932196609747662866401954341497716363174035275761,
    194: 17829232438088717947765479975403534458051445077992682449561,
    196: 65066066269192108839947861329734188452463615549935803519143,
    198: 349950642518780039677569542727000637357372102125566743672291,
    200: 1292650905825941096257239453478790385594125558306176183893071,
    202: 5212568709703227409964986200323079271990196603993590694680391,
    204: 24456849430211698827484825800200226142835068670603824696509853,
    206: 86484458957105177897678864034678638961222781895382346786419167,
    208: 282008119289803047168077576239866793179892385097797258038424567,
    210: 1204782513175327651249234385485579508935520775091508304137907593,
    212: 4557491279500788785092872521598135415310018444227180886738032603,
    214: 20157487867138959990956400818670113252626858623502858096773250117,
    216: 76043131885256815931216222672122360339414261530624664568946162829,
    218: 344050409069283777519808151926058652189426586409858940845468981861,
    220: 1261954136810361898882562305061560803960332696773429051013499670427,
    222: 4967098631546029459529052889119752620821846506121675016252303705879,
    224: 16820438383635236108780123377884102235161339126337922957523995233529,
    226: 89004872228161109668642757451109313028486456927748001177934114653617,
    228: 417997545002926789713327767706388156900167929618181881789066408078713,
    230: 1448119400401161666430996998583433602188054714520957809331008374203811,
    232: 6085700232501070308789059668383263517284617763227080242662586334468949,
    234: 18761475251108592243772325275525370672745869289561231704528591592371501,
    236: 75378971952862314631435911840791163346363206779933878820532663796306607,
    238: 334251059867417571197273497079020101091157708427514612078154549573744053,
    240: 1293224958516444312449612523948303217126432531171026209139782551333088281,
    242: 5936906321010438090569400987846435049778779894646896974880143196051640311,
    244: 20576188160125558108332221675210966458468072966170808188722656711461127553,
    246: 80363125512735611009514922977138951394216679592881179768811803231929707129,
    248: 371791977264476193233793691135938456442939709119625615853629510058608968387,
    250: 1042050145513045807703000231691018818256527018925081399314153802338188332967,
    252: 5279774839958419063483802629238621577728731461590896999443064285556245912531,
    254: 20335945572378210037443669733175394361807801065230296579184489046120469661507,
    256: 104343749483876530238735961449384774965065842501756473864398652143393515842787,
}


def qft_dagger(qc: QuantumCircuit, n: int) -> None:
    """
    Apply the inverse Quantum Fourier Transform (QFT) to the quantum circuit.

    :param qc: Quantum circuit object
    :param n: Number of qubits
    """
    # Swap symmetric qubits positions
    for i in range(n // 2):
        qc.swap(i, n - i - 1)
    # Apply controlled phase rotations and Hadamard gates in reverse order
    for i in range(n):
        for j in range(i):
            angle = -math.pi / (2 ** (i - j))
            qc.cp(angle, j, i)
        qc.h(i)


def solve_factors(N: int):
    """
    Solve the factors of N using a simple brute-force search;
    if factors are found, return (p, q), otherwise return (None, None).
    """
    for p in range(2, int(math.sqrt(N)) + 1):
        if N % p == 0:
            q = N // p
            return p, q
    return None, None


def shors_algorithm(N: int):
    """
    Simulate the factorization of N using a simplified version of Shor's algorithm.
    If N is even, return (2, N/2) directly.
    """
    # Fast factorization for even numbers
    if N % 2 == 0:
        return 2, N // 2

    # Choose a random integer a such that gcd(a, N) == 1
    a = random.randint(2, N - 1)
    max_attempts = 100  # Set a maximum number of attempts to avoid an infinite loop
    attempts = 0
    while gcd(a, N) != 1 and attempts < max_attempts:
        a = random.randint(2, N - 1)
        attempts += 1
    if gcd(a, N) != 1:
        print(f"Unable to find an integer a that is coprime with {N}, current a={a}")
        return None, None

    print(f"Factorizing N={N}, chosen a={a}")

    # Define the number of qubits for the quantum circuit
    num_qubits = math.ceil(math.log2(N)) + 1
    qc = QuantumCircuit(num_qubits, num_qubits)

    # Apply Hadamard gates to all qubits
    qc.h(range(num_qubits))

    # Simulate modular exponentiation (this is just a placeholder, can be replaced with a more detailed implementation as needed)
    qc.cx(0, 1)
    qc.cx(1, 2)
    qc.barrier()

    # Apply inverse QFT
    qft_dagger(qc, num_qubits)

    # Measure
    qc.measure(range(num_qubits), range(num_qubits))

    # Execute the quantum circuit
    # Use the backend provided by QuantumRings
    mybackend = QrBackendV2(provider, num_qubits=num_qubits)
    qc_transpiled = transpile(qc, mybackend, initial_layout=list(range(qc.num_qubits)))

    gate_count = qc_transpiled.count_ops()
    print("Gate Count:", gate_count, "Total Gates:", sum(gate_count.values()))

    job = mybackend.run(qc_transpiled, shots=1024)
    result = job.result()
    counts = result.get_counts()
    print("Measurement results:", counts)

    # Extract values from measurement results (currently a simple approach: iterate over all even results)
    measured_values = [int(bit_str, 2) for bit_str in counts.keys()]
    for r in measured_values:
        if r % 2 == 1:
            continue  # Only consider even results
        p, q = solve_factors(N)
        if p and q:
            print(f"Factors found: {p}, {q}")
            return p, q

    print("Failed to factorize.")
    return None, None


if __name__ == "__main__":
    # Test factorization for all semiprime numbers
    for bits in sorted(semiprimes.keys()):
        N = semiprimes[bits]
        print(f"\nFactorizing {N} (bit length: {bits})")
        factors = shors_algorithm(N)
        if factors:
            print(f"Success: {N} = {factors[0]} × {factors[1]}")
        else:
            print(f"Failed to factorize {N}.")
